# Experimentos: Baseline com SVM Clássico

Este notebook estabelece baselines de performance com um modelo de Support Vector Machine (SVM) clássico em datasets

# Dataset = Breast Cancer Wisconsin (Diagnostic)

In [1]:
from sklearn.datasets import load_breast_cancer 
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC  # Importando o classificador SVM
from sklearn.metrics import classification_report
import numpy as np
import pandas as pd

## Loading data and initial analysis

In [2]:
X, y = load_breast_cancer(return_X_y=True)  # Carregando o dataset

# Verificando as dimensões dos nossos dados
print(f"Dimensões dos dados: {X.shape}")


# Vendo a distribuição das classes (0 e 1)
print("\nDistribuição das classes:")
# Usando numpy para contar as ocorrências de cada classe
unique, counts = np.unique(y, return_counts=True)
for class_label, count in zip(unique, counts):
    print(f"Classe {class_label}: {count} ocorrências")


Dimensões dos dados: (569, 30)

Distribuição das classes:
Classe 0: 212 ocorrências
Classe 1: 357 ocorrências


## Split data into training and testing sets

In [3]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42, stratify=y) # Stratify -> mantém a proporção das classes no split

print(f"Dimensões do conjunto de treino: {X_train.shape}")
print(f"Dimensões do conjunto de teste: {X_test.shape}")


Dimensões do conjunto de treino: (426, 30)
Dimensões do conjunto de teste: (143, 30)


## Normalize data

In [4]:
# normalizing the data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)  # Ajusta o scaler e transforma os dados de treino
X_test = scaler.transform(X_test)  # Transforma os dados de teste com o mesmo scaler

print("\nDados normalizados")






Dados normalizados


# Training SVM model

In [5]:
# Treinando o modelo SVM
model = SVC(kernel="linear", gamma="scale", random_state=42)  # Usando o kernel RBF
model.fit(X_train, y_train)

print("\nModelo treinado com SVM (kernel linear)")


Modelo treinado com SVM (kernel linear)


# Predictions

In [6]:
# predicting the test set
predictions = model.predict(X_test) # Prevendo os rótulos do conjunto de teste
print("\nPrevisões feitas no conjunto de teste")
# Avaliando o modelo
print("\nRelatório de Classificação:")
print(classification_report(y_test, predictions))


Previsões feitas no conjunto de teste

Relatório de Classificação:
              precision    recall  f1-score   support

           0       0.98      0.98      0.98        53
           1       0.99      0.99      0.99        90

    accuracy                           0.99       143
   macro avg       0.99      0.99      0.99       143
weighted avg       0.99      0.99      0.99       143



# Dataset = Student Performance(POR and MAT)

In [ ]:
from imblearn.over_sampling import SMOTE

def run_svm_experiment(filepath, subject_name, separator=','):
    """
    Carrega um dataset de performance de alunos, executa o pré-processamento
    e treina um modelo SVM clássico, retornando o relatório de classificação.
    
    Args:
        filepath (str): O caminho para o arquivo .csv.
        subject_name (str): O nome da matéria para os printouts (ex: 'Math').
    """
    
    print(f"Iniciando processamento para: {subject_name} ")
    
    # Carregamento e Pré-processamento 
    df = pd.read_csv(filepath, sep=separator)
    print(f"Formato: {df.shape}")
    y_grades = df[['G1', 'G2', 'G3']]
    X_features = df.drop(columns=['G1', 'G2', 'G3'])

    
    y_final = np.where(y_grades['G3'] >= 10, 1, 0)
    print("\nDistribuição de classes (0=Reprovado, 1=Aprovado):")
    print(pd.Series(y_final).value_counts())

    X_final = pd.get_dummies(X_features, drop_first=True)
    print(f"\nFormato após one-hot encoding: {X_final.shape}")

    # Divisão, Normalização e SMOTE
    X_train, X_test, y_train, y_test = train_test_split(
        X_final, y_final, test_size=0.25, random_state=42, stratify=y_final
    )
    print(f"\nDados divididos: {X_train.shape[0]} para treino, {X_test.shape[0]} para teste.\n")

    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    
    
    print("\nDistribuição de classes no treino ANTES do SMOTE:")
    print(pd.Series(y_train).value_counts())
    
    smote = SMOTE(random_state=42)
    X_train_resampled, y_train_resampled = smote.fit_resample(X_train_scaled, y_train)
    
    print("\nDistribuição de classes no treino DEPOIS do SMOTE:")
    print(pd.Series(y_train_resampled).value_counts())

    # Treinamento e Avaliação
    svm_model = SVC(kernel='rbf', gamma='scale', random_state=42)
    svm_model.fit(X_train_resampled, y_train_resampled)
    
    predictions = svm_model.predict(X_test_scaled)
    
    print(f"\n   Relatório de Classificação para SVM ({subject_name})    \n")
    print(classification_report(y_test, predictions, target_names=['Reprovado', 'Aprovado']))

# Execution

In [8]:

path_portugues = '../data/student-por.csv'
path_matematica = '../data/student-mat.csv'


run_svm_experiment(filepath=path_portugues, subject_name="Português", separator=",")

print("\n" + "="*50 + "\n") 


run_svm_experiment(filepath=path_matematica, subject_name="Math", separator=";")

Iniciando processamento para: Português 
Formato: (649, 33)

Distribuição de classes (0=Reprovado, 1=Aprovado):
1    549
0    100
Name: count, dtype: int64

Formato após one-hot encoding: (649, 39)

Dados divididos: 486 para treino, 163 para teste.


Distribuição de classes no treino ANTES do SMOTE:
1    411
0     75
Name: count, dtype: int64

Distribuição de classes no treino DEPOIS do SMOTE:
1    411
0    411
Name: count, dtype: int64

   Relatório de Classificação para SVM (Português)    

              precision    recall  f1-score   support

   Reprovado       0.36      0.40      0.38        25
    Aprovado       0.89      0.87      0.88       138

    accuracy                           0.80       163
   macro avg       0.62      0.63      0.63       163
weighted avg       0.81      0.80      0.80       163



Iniciando processamento para: Math 
Formato: (395, 33)

Distribuição de classes (0=Reprovado, 1=Aprovado):
1    265
0    130
Name: count, dtype: int64

Formato após one-hot 